# Vector Fitting Examples

The Vector Fitting feature is demonstrated using to three example networks from the scikit-rf `data` folder. Additional explanations and background information can be found in the Vector Fitting tutorial.

In [ ]:
import skrf
import numpy as np

## Example 1: Ring Slot

To create a `VectorFitting` instance, a `Network` containing the frequency responses of the N-port is passed. In this first example the *ring slot* is used:

In [ ]:
nw1 = skrf.data.ring_slot
vf1 = skrf.VectorFitting(nw1)

Now, the vector fit can be performed. The number of poles has to be specified, which depends on the *behaviour* of the responses. A smooth response would only require very few poles (2-5). In this case, 2 real poles are sufficient:

In [ ]:
vf1.vector_fit(n_poles_real=2, n_poles_cmplx=0)

As printed in the logging output (not shown), the pole relocation process converged quickly after just 4 iteration steps. The fitted model parameters are now stored in the class variables `poles`, `zeros`, `proportional_coeff` and `constant_coeff` for further use. To verify the result, the model response can be compared to the original network response. As the model will return a response at any given frequency, it makes sense to also check its response outside the frequency range of the original samples:

In [ ]:
freqs1 = np.linspace(0, 200e9, 201)
vf1.plot_s_mag(0, 0, freqs1) # plot s11
vf1.plot_s_mag(1, 0, freqs1) # plot s21

To use the model in a circuit simulation, an equivalent circuit can be created based on the fitting parameters. This is currently only implemented for SPICE, but the structure of the equivalent circuit can be adopted to any kind of circuit simulator.

`vf1.write_spice_subcircuit_s('/home/vinc/Desktop/ring_slot.sp.sp')`

For a quick test, the subcircuit is included in a schematic in [QUCS-S](https://ra3xdh.github.io/) for AC simulation and S-parameter calculation based on the port voltages and currents (see the equations):
<img src="./ngspice_ringslot_schematic.svg" />

The simulation outputs from [ngspice](http://ngspice.sourceforge.net/) compare well to the plots above:
<img src="./ngspice_ringslot_sp_mag.svg" />
<img src="./ngspice_ringslot_sp_smith.svg" />

## Example 2: Measured 190 GHz Active Circuit

This example is a lot more tricky to fit, because the responses contain a few "bumps" and noise from the measurement. In such a case, finding a good number of initial poles can take a few iterations. 

Load the Network from a Touchstone file and create the Vector Fitting instance:

In [ ]:
nw2 = skrf.network.Network('./190ghz_tx_measured.S2P')
vf2 = skrf.VectorFitting(nw2)

**First attempt:** Perform the fit using 4 real poles and 3 complex-conjugate poles with *linear* spacing (default):

In [ ]:
vf2.vector_fit(n_poles_real=4, n_poles_cmplx=3)

The function `plot_convergence()` can be helpful to examine the convergence and see if something was going wrong. In this case, it took quite a while (84 iteration steps), but the results converged nevertheless.

In [ ]:
vf2.plot_convergence()

Checking the results by comparing the model responses to the original sampled data indicates a successful fit:

In [ ]:
vf2.plot_s_mag(0, 0) # s11
vf2.plot_s_mag(1, 0) # s21

It is a good idea to also check the model response well outside the original frequency range. This reveals a strong resonance at higher frequencies (at approx. 330 GHz), which is not ideal:

In [ ]:
freqs2 = np.linspace(0, 500e9, 501)
vf2.plot_s_mag(0, 0, freqs2)
vf2.plot_s_mag(1, 0, freqs2)

**Second attempt:** Maybe an even better fit can be achieved, so let's try again. It sometimes helps to change the initial pole spacing from *linear* to *logarithmic*:

In [ ]:
vf2.vector_fit(n_poles_real=4, n_poles_cmplx=3, init_pole_spacing='log')
vf2.plot_convergence()

This fit converged slightly quicker (66 iteration steps) and also matches the network data very well inside the fitting band. Interestingly, the strong resonance from before in the outside band is replaced with a much weaker one at 263 GHz:

In [ ]:
vf2.plot_s_mag(0, 0)
vf2.plot_s_mag(1, 0)
vf2.plot_s_mag(0, 0, freqs2)
vf2.plot_s_mag(1, 0, freqs2)

This looks good, so let's export the model as a SPICE subcircuit. For example:

`vf2.write_spice_subcircuit_s('/home/vinc/Desktop/190ghz_tx.sp')`

The subcircuit can then be simulated in SPICE as before in [example 1](#Example-1:-Ring-Slot):
<img src="./ngspice_190ghz_tx_sp_mag.svg" />
<img src="./ngspice_190ghz_tx_sp_smith.svg" />